Szöveg generálása, Stephen King - The Eyes of the Dragon könyve alapján

Mercs Alex - JESJCD



Első lépésben kisbetűssé alakítjuk a teljes szöveget, hogy kezelhetőbb legyen

In [5]:
with open("Stephen_King_The_Eyes_Of_The_Dragon.txt", "r", encoding="utf8") as book:
    script = book.read().lower()

#len() fgv megszámolja a karakterek számát
print(len(script))

569829


In [ ]:
#script[:569829] - teljes szöveg megjelenítése

## Szöveg karaktereinek kigyűjtése

numpy: python matematikai függvénykönyvtára

tqdm: vizualizálást tesz lehetővé

In [6]:
import numpy as np
from tqdm import tqdm

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in tqdm(range(0, len(script) - maxlen, step)):
    sentences.append(script[i: i + maxlen])
    next_chars.append(script[i + maxlen])
len(sentences)

100%|██████████| 189923/189923 [00:00<00:00, 1052819.47it/s]


189923

Előforduló karakterek kiírása:

In [7]:
chars = sorted(list(set(script)))
print(chars)

['\n', ' ', '!', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '5', '6', '7', '8', '9', ':', ';', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '¬', '—', '‘', '’', '“', '”']


Ezen karakterek indexei:

In [8]:
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '&': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, '_': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49, '}': 50, '¬': 51, '—': 52, '‘': 53, '’': 54, '“': 55, '”': 56}


In [9]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("x", x.shape)
print("y", y.shape)

189923it [00:04, 41399.07it/s]

x (189923, 60, 57)
y (189923, 57)


## Model tanítása:
A tensorflow modellek tanítására használatos eszköz.

In [10]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.GRU(16, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 16)                3600      
                                                                 
 dense (Dense)               (None, 57)                969       
                                                                 
Total params: 4569 (17.85 KB)
Trainable params: 4569 (17.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


hihetőbb eredmény érdekében megnövelem az epochs számát

In [21]:
with tf.device("/CPU:0"):
    model.fit(x,
            y,
            batch_size=256,
            epochs=10)

Epoch 1/10
742/742 [==============================] - 37s 50ms/step - loss: 1.8193
Epoch 2/10
742/742 [==============================] - 37s 50ms/step - loss: 1.8186
Epoch 3/10
742/742 [==============================] - 37s 50ms/step - loss: 1.8193
Epoch 4/10
742/742 [==============================] - 37s 50ms/step - loss: 1.8176
Epoch 5/10
742/742 [==============================] - 37s 50ms/step - loss: 1.8165
Epoch 6/10
742/742 [==============================] - 37s 50ms/step - loss: 1.8154
Epoch 7/10
742/742 [==============================] - 38s 51ms/step - loss: 1.8138
Epoch 8/10
742/742 [==============================] - 38s 51ms/step - loss: 1.8128
Epoch 9/10
742/742 [==============================] - 38s 51ms/step - loss: 1.8120
Epoch 10/10
742/742 [==============================] - 38s 51ms/step - loss: 1.8108


Kiindulópontnak veszünk egy random sort. Generálás folyamán mindig ezzel a sorral kezdődik majd a szöveg

In [22]:
import random

start_index = random.randint(0, len(script) - maxlen - 1)
base_text = script[start_index: start_index + maxlen]
base_text

'been \ndriven from the castle keep from the very kingdom itse'

In [23]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Generálás
Végül legenerálhatjuk a kívánt hosszúságú szöveget

In [26]:
import sys
temperatures = [0.2, 0.5, 1.0, 1.2] #0.2 kicsit stroke gyanús, 0.5 tűrhető, a további értékeknék Shakespear által írt szövegnek tűnik, csak nem létező szavakkal
generated_characters = 200

for temp in temperatures:
    print("Temp: ", temp)
    generated_text = base_text
    print(generated_text)
    for i in range(generated_characters):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]

        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

    print()

Temp:  0.2
been 
driven from the castle keep from the very kingdom itse
d the 
the been the stin the 
his he he he pery the sought the stire the not the the the the the 
he and the the stite he and he the been the the was the the the was and whit the the for the 
with and
Temp:  0.5
been 
driven from the castle keep from the very kingdom itse
d the and he of the your still and he the murs the on the say mashed mane 
her the she his been of his and come he the 
you and the 




stares for he the momen the sece 
denning 
all the of strears 

Temp:  1.0
been 
driven from the castle keep from the very kingdom itse
nt 
reen mind take one last medrate on a greatt wanes, scaeding crough she king. if the ke stoods devernen’s saidleres of as suron, 
you boodn’t menetts hay. “and werlepsong knets 
cont. “bit grap. 


Temp:  1.2
been 
driven from the castle keep from the very kingdom itse
ncing nosher everking sindy outss, frome, all, notnving bagh-take and squary itesssomed baad climt. 

“edncou